In [ ]:
import pandas as pd
import math

In [ ]:
import math
import pandas as pd

def hitung_jarak(x, y, a, b):
    jarak = math.sqrt((x - a)**2 + (y - b)**2)
    return jarak

def tentukan_jaringan(jarak):
    if jarak <= 11:
        return 'kuat (hijau)'
    else:
        return 'lemah (kuning)'

def calculate_t(x0, y0, x1, y1, p, q, r, s):
    denominator = (x1 - x0) * r - (y1 - y0) * p

    if denominator != 0:
        numerator = (y1 - y0) * q - (y1 - y0) * x0 - (x1 - x0) * s + (x1 - x0) * y0
        return numerator / denominator
    else:
        return None

def calculate_t2(x2, y2, s0, q0, p0, r0):
    denominator = p0 - r0

    if denominator != 0:
        numerator = x2 - y2 + s0 - q0
        return numerator / denominator
    else:
        return None

def hitung_jumlah_titik_hijau(df_j, df_t, hitung_t21=True, hitung_t22=False):
    jumlah_titik_hijau = 0

    if hitung_t21:
        jumlah_titik_hijau += ((2 * df_j['jarak']) - (df_t['t2'] * df_j['jarak']) <= 11).sum()

    if hitung_t22:
        if 't22' in df_t.columns:
            jumlah_titik_hijau += ((2**2 * df_j['jarak']) - (2**1 * df_t['t22'] * df_j['jarak']) - (2**0 * df_t['t21'] * df_j['jarak']) <= 11 ).sum()

    return jumlah_titik_hijau


In [ ]:
import pandas as pd

def ALLIN(df_titik, df_access_point):
    pd.set_option('display.float_format', lambda x: '{:.5f}'.format(x))

    hasil_list = []

    for index, row in df_titik.iterrows():
        x_titik_layanan = row['a']
        y_titik_layanan = row['b']

        x_access_point_terdekat = None
        y_access_point_terdekat = None
        jarak_minimal = float('inf')

        for idx, rw in df_access_point.iterrows():
            jarak = hitung_jarak(rw['x'], rw['y'], x_titik_layanan, y_titik_layanan)

            if jarak < jarak_minimal:
                x_access_point_terdekat = rw['x']
                y_access_point_terdekat = rw['y']
                jarak_minimal = jarak

        kategori_jaringan = tentukan_jaringan(jarak_minimal)

        hasil_list.append({'x1': x_access_point_terdekat,
                            'y1': y_access_point_terdekat,
                            'x0': x_titik_layanan,
                            'y0': y_titik_layanan,
                            'Jarak': jarak_minimal,
                            'Kategori Jaringan': kategori_jaringan})

    df_hasil_terdekat = pd.DataFrame(hasil_list)
    df_hasil_terdekat = df_hasil_terdekat.loc[(df_hasil_terdekat['Jarak'] <= 11) |
                                               (df_hasil_terdekat['Kategori Jaringan'] != 'lemah (kuning)')]

    data1 = df_hasil_terdekat
    garispqrs = 'https://raw.githubusercontent.com/Riyyan16/Data-TA/master/garis%20persamaan%20(versi%20pqrs).xlsx'
    data2 = pd.read_excel(garispqrs)

    results = []

    for index1, row1 in data1.iterrows():
        for index2, row2 in data2.iterrows():
            t = calculate_t(row1['x0'], row1['y0'], row1['x1'], row1['y1'],
                            row2['p'], row2['q'], row2['r'], row2['s'])
            if t is None:
                continue
            if 0 <= t <= 1:
                results.append({'x0': row1['x0'], 'y0': row1['y0'], 'x1': row1['x1'], 'y1': row1['y1'],
                                'p': row2['p'], 'q': row2['q'], 'r': row2['r'], 's': row2['s'], 't': t})

    result_df = pd.DataFrame(results)
    result_df['x2'] = result_df['p']*result_df['t'] + result_df['q']
    result_df['y2'] = result_df['r']*result_df['t'] + result_df['s']
    result_df['q0'] = result_df['x1']
    result_df['s0'] = result_df['y1']
    result_df['p0'] = result_df['x0'] - result_df['x1']
    result_df['r0'] = result_df['y0'] - result_df['y1']
    result_df['jarak'] = ((result_df['x1'] - result_df['x0'])**2 + (result_df['y1'] - result_df['y0'])**2)**0.5

    results = []
    for index, row in result_df.iterrows():
        t = calculate_t2(row['x2'], row['y2'], row['s0'], row['q0'], row['p0'], row['r0'])
        if t is None:
            continue
        if 0 <= t <= 1:
            results.append({'x2': row['x2'], 'y2': row['y2'], 's0': row['s0'], 'q0': row['q0'],
                            'p0': row['p0'], 'r0': row['r0'], 't2': t})

    t2_df = pd.DataFrame(results)

    t22 = t2_df[t2_df.duplicated(subset=["y2","x2"], keep=False)].sort_values('t2')

    jumlah_titik_t21 = hitung_jumlah_titik_hijau(result_df, t2_df, hitung_t21=True, hitung_t22=False)
    jumlah_titik_t22 = hitung_jumlah_titik_hijau(result_df, t2_df, hitung_t21=False, hitung_t22=True)

    return jumlah_titik_t21 + jumlah_titik_t22


# Define the necessary functions: hitung_jarak, tentukan_jaringan, calculate_t, calculate_t2
# Make sure these functions are implemented and available

# Load data from URLs
urlAP = 'https://raw.githubusercontent.com/Riyyan16/Data-TA/master/titik%20koordinat%20access%20pont%201-7.xlsx'
urlKoor = 'https://raw.githubusercontent.com/Riyyan16/Data-TA/master/Titik%20Koordinat%20Pemetaan%20AP.xlsx'

df_access_point = pd.read_excel(urlAP)
df_titik = pd.read_excel(urlKoor)


ALLIN(df_titik, df_access_point)

693

In [ ]:
import numpy as np
import pandas as pd
import random
import math
import matplotlib.pyplot as plt

urlKoor = 'https://raw.githubusercontent.com/Riyyan16/Data-TA/master/Titik%20Koordinat%20Pemetaan%20AP.xlsx'
df_titik = pd.read_excel(urlKoor)

# Fungsi Fitness
def fitness(population):
    fitness_values = []
    jumlah_titik = []
    for ind in population:
        df = pd.DataFrame(ind, columns=['x', 'y'])
        jumlahTitik = ALLIN(df_titik, df)
        fitness_values.append(jumlahTitik / 1240)
        jumlah_titik.append(jumlahTitik)
    print("jumlah titik:", jumlah_titik)
    return fitness_values

# Fungsi crossover (persilangan)
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
    child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
    return child1, child2

# Fungsi mutasi
def mutate(individual, mutation_rate):
    for i in range(len(individual)):
        if np.random.rand() < mutation_rate:
            individual[i] = np.random.uniform(0, max_coordinate)
    return individual

# Algoritma Genetika
def genetic_algorithm(population_size, generations, mutation_rate):
    # Ambil koordinat dari df_titik
    coordinates_from_df = df_titik[['a', 'b']].values.tolist()

    # Inisialisasi populasi awal secara acak dengan koordinat dari df_titik
    population = [np.array(random.sample(coordinates_from_df, 7)) for _ in range(population_size-1)]

    # Coordinates to add
    coordinates_to_add = [
                          [29.9, 9.6],
                          [52.65, 9.6],
                          [9.05, 24.65],
                          [20.85, 28.8],
                          [41.25, 28.6],
                          [59.6, 29.1],
                          [69.7, 24.72]
                          ]

    # Adding coordinates to the population
    population += [np.array(coordinates_to_add)]
    best_individual = []
    fitness_values = 0
    for generation in range(generations):
        # Menghitung nilai fitness
        fitness_values = fitness(population)

        # Memilih orang tua (parent) dengan seleksi turnamen
        tournament_size = 5
        parents = []
        for _ in range(population_size):
            tournament_indices = np.random.choice(population_size, tournament_size, replace=False)
            tournament_fitness = [fitness_values[i] for i in tournament_indices]
            selected_parent_index = tournament_indices[np.argmax(tournament_fitness)]
            parents.append(population[selected_parent_index])

        # Melakukan crossover untuk menghasilkan anak
        children = []
        for i in range(0, population_size, 2):
            child1, child2 = crossover(parents[i], parents[i + 1])
            children.extend([child1, child2])

        # Melakukan mutasi pada anak
        mutated_children = [mutate(child, mutation_rate) for child in children]

        # Menggantikan populasi lama dengan anak-anak yang baru
        population = mutated_children

        # Menampilkan nilai fitness terbaik setiap generasi (opsional)
        best_fitness = max(fitness_values)
        print(f"Generation {generation + 1}, Best Fitness: {best_fitness}")

        # Mengembalikan individu terbaik (penempatan access point terbaik)
        best_individual_index = np.argmax(fitness_values)
        best_individual = population[best_individual_index]
        print("Best Access Point Placement:")
        print(best_individual)

    return best_individual

# Koordinat titik dalam area penempatan access point
coordinates = [(x, y) for x in range(79) for y in range(32)]

# Batas koordinat maksimum
max_coordinate = 79, 32

# Menentukan parameter algoritma genetika
population_size = 10
generations = 30
mutation_rate = 0.1

# Menjalankan algoritma genetika
best_placement = genetic_algorithm(population_size, generations, mutation_rate)

# Menampilkan hasil penempatan access point terbaik
print("Best Access Point Placement:")
print(best_placement)

# Menampilkan koordinat setiap access point dari individu terbaik
for i, coordinates in enumerate(best_placement):
    x, y = coordinates
    print(f"Access Point {i+1}: ({x}, {y})")


jumlah titik: [521, 281, 455, 468, 404, 734, 763, 515, 598, 693]
Generation 1, Best Fitness: 0.6153225806451613
Best Access Point Placement:
[[68 31]
 [31  4]
 [52 27]
 [44  7]
 [30 18]
 [42  1]
 [33 22]]
jumlah titik: [743, 315, 679, 641, 722, 514, 763, 476, 762, 518]
Generation 2, Best Fitness: 0.6153225806451613
Best Access Point Placement:
[[68. 31.]
 [31.  4.]
 [52. 27.]
 [44.  7.]
 [30. 18.]
 [42.  1.]
 [33. 22.]]
jumlah titik: [787, 633, 797, 787, 577, 794, 763, 732, 774, 731]
Generation 3, Best Fitness: 0.6427419354838709
Best Access Point Placement:
[[29.9  9.6]
 [31.   4. ]
 [52.  27. ]
 [44.   7. ]
 [30.  18. ]
 [42.   1. ]
 [33.  22. ]]
jumlah titik: [774, 797, 726, 704, 669, 786, 813, 665, 726, 772]
Generation 4, Best Fitness: 0.6556451612903226
Best Access Point Placement:
[[29.9         9.6       ]
 [31.          4.        ]
 [52.         27.        ]
 [44.          7.        ]
 [30.         18.        ]
 [65.         24.        ]
 [ 1.40376274 27.00789569]]
jumlah titik

In [ ]:
def plot_fitness_progress(fitness_progress):
    # Menampilkan grafik perkembangan fitness
    plt.figure(figsize=(10, 6))
    plt.plot(fitness_progress, marker='o', color='b')
    plt.title('Fitness Progress')
    plt.xlabel('Generation')
    plt.ylabel('Fitness')
    plt.grid(True)
    plt.show()
    plot_fitness_progress(fitness_progress)
    plot_fitness_progress(fitness_progress)


In [ ]:
import numpy as np

In [ ]:
np.random.choice(10, 3, replace=False)

NameError: name 'np' is not defined

In [ ]:
np.random.uniform(0, 1024)

739.8805585778036